In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Thursday, October 5th, 5:00 PM

## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

You may find this project's queries to be simpler than in Project 1. However, although the queries may not be as complex, we still expect you to spend ample time thinking through the effects of each of the methods, as reasoning about the tradeoff between different approaches is the goal of this assignment.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Logistics & Scoring Breakdown

- Each coding question has **both public tests and hidden tests**. Roughly 50% of your coding grade will be made up of your score on the public tests released to you, while the remaining 50% will be made up of unreleased hidden tests.
- Public tests for multiple choice questions are for sanity check only (e.g. you are answering in the correct format). Partial credit will be awarded.
- Free-response questions will be manually graded. Please answer thoughtfully and concisely in complete sentences, drawing from knowledge in lectures and from your inspection of query plans.

This is an **individual project**. However, you’re welcome to collaborate with any other student in the class as long as it’s within the academic honesty guidelines.


| Question    | 0 | 1 | 2              | 3    | 4    | 5              | 6              | 7    | 8              | 9        | 10        |
| ----------- | - | - | -------------- | ---- | ---- | -------------- | -------------- | ---- | -------------- | -------- | --------- |
| No Subparts | 1 |   |                |      |      |                |                |      |                |          | 6         |
| a           |   | 1 | 1              | 1    | 1    | 2              | 2              | 2    | 1              | 1        |           |
| b           |   | 3 | 3              | 1    | 1    | 1              | 2              | 1    | 1              | 1        |           |
| c           |   |   | 1              | m: 2 | 1    | 1              | 1              | m: 3 | 1              | 2 (m: 2) |           |
| d           |   |   | 4 (m: 2, a: 2) | m: 2 | m: 3 | 4 (m: 2, a: 2) | 1              |      | 4 (m: 2, a: 2) |          |           |
| e           |   |   |                |      |      |                | 4 (m: 2, a: 2) |      |                |          |           |
| **Total**       | 1 | 4 | 9              | 6    | 6    | 8              | 10             | 6    | 7              | 4        | manual: 6 |


**Grand Total:** 67 points (manual: 26, autograded: 41)

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `JupySQL` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

In [4]:
# See full display
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

**Note:** If you run into the **role does not exist**/**database does not exist** error the first time you run this cell, feel free to ignore it. It does not affect data import.

In [5]:
!unzip -u data/baseball.zip -d data/

Archive:  data/baseball.zip


In [6]:
!psql postgresql://jovyan@127.0.0.1:5432/baseball -c 'SELECT pg_terminate_backend(pg_stat_activity.pid) FROM pg_stat_activity WHERE datname = current_database()  AND pid <> pg_backend_pid();'
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!psql -h localhost -d baseball -f data/baseball.sql
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

 pg_terminate_backend 
----------------------
(0 rows)

DROP DATABASE
CREATE DATABASE
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
CREATE TABLE
ALTER TABLE
COPY 5219
COPY 104256
COPY 179
COPY 6236
COPY 425
COPY 6879
COPY 104324
COPY 13943
COPY 17350
COPY 138838
COPY 12028
COPY 31955
COPY 13110
COPY 4191
COPY 3040

Running query in 'postgresql://jovyan@127.0.0.1:5432/postgres'

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb,jovyan,UTF8,en_US.utf8,en_US.utf8,None
imdb_lecture,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [7]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

Connecting and switching to connection postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [8]:
%%sql
SELECT * FROM halloffame LIMIT 5

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

5 rows affected.

playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


## Connect to the grader

Run the following cell for grading purposes.

In [9]:
# Just run the following cell, no further action is needed.
from data101_utils import GradingUtil
grading_util = GradingUtil("proj2")
grading_util.prepare_autograder()

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

As a reminder from Project 1, `%sql \d <table_name>` is helpful for identifying the columns in a table.

<br><br>
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Question 0: PostgreSQL Explain Analyze
**Please read through this section carefully, as a vast majority of the project will require you to inspect query plans via interpreting the output of the explain analyze command.**

To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_query_str = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_query_str"`

Take a look at the following sample query plan.

![title](data/sample_query.png)

It is highly recommended to read through [this article](https://www.cybertec-postgresql.com/en/how-to-interpret-postgresql-explain-analyze-output/) and the postgreSQL [documentation 14.1.2](https://www.postgresql.org/docs/current/using-explain.html#USING-EXPLAIN-ANALYZE) to see how you can interpret the output above. Everything before "Tools to interpret Explain Analyze output" is useful.


<div class="alert alert-block alert-info">
Here are some key things to note for all question parts:
<ul>
<li>When we ask you to identify the <b>query cost</b>, we are looking for the <b>total cost</b>.</li>
    <ul>
    <li>There are two cost values: the first is the <b>startup cost</b> (cost to return the first row) and the second is the <b>total cost</b> (cost to return all rows).</li>
    <li>The unit for the estimated query cost is an arbitrary estimation of disk I/O (1 is the cost for reading an 8kB page during a sequential scan).</li>
        <li>Feel free to round the query cost / time to the nearest integer, but we'll accept anything more exact.</li>
    </ul>
<li>When we ask you to identify the <b>query time</b>, we are looking for the <b>execution time</b> (in ms).</li>
    <ul>
        <li>We recognize that the execution time may vary between different cell executions, so the autograder will tolerate a reasonable range.</li>
    </ul>
</ul>
</div>

Now, inspect the query plan above by following the below steps:

1. Manually copy the entire query command (i.e., `SELECT ... `) from the screenshot into the cell below.

_Type your answer here, replacing this text._

In [10]:
%%sql --save query_0 result_0 <<
SELECT * FROM people AS p INNER JOIN collegeplaying AS cp ON p.playerid = cp.playerid

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

In [11]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_0 = %sqlcmd snippets query_0
grading_util.save_results("result_0", query_0, result_0);
result_0.DataFrame().head(3)

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,height,bats,throws,debut,finalgame,retroid,bbrefid,playerid,schoolid,yearid
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


In [12]:
%sql EXPLAIN ANALYZE {{query_0}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

7 rows affected.

QUERY PLAN
Merge Join (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.045..16.062 rows=17350 loops=1)
Merge Cond: ((p.playerid)::text = (cp.playerid)::text)
-> Index Scan using master_pkey on people p (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.021..2.925 rows=19368 loops=1)
-> Index Only Scan using collegeplaying_pkey on collegeplaying cp (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.018..3.777 rows=17350 loops=1)
Heap Fetches: 17350
Planning Time: 0.123 ms
Execution Time: 16.581 ms


In [13]:
sample_query_cost =1193.88
sample_query_timing = 23.370

In [14]:
grader.check("q0")

q0 results: All test cases passed!

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 1: Queries and Views, Part 1

In Questions 1 and 2, you will compare and contrast writing queries with subqueries and views.

## Question 1a
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **NOT** use any sub-queries.

In [15]:
%%sql --save query_1a result_1a <<
SELECT p.namefirst, p.namelast, p.playerid, h.yearid
FROM people AS p
INNER JOIN halloffame AS h ON p.playerid = h.playerid
WHERE h.inducted = 'Y'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

323 rows affected.

In [16]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_1a = %sqlcmd snippets query_1a
grading_util.save_results("result_1a", query_1a, result_1a)
result_1a.DataFrame().head(3)

,namefirst,namelast,playerid,yearid
0,Ty,Cobb,cobbty01,1936
1,Babe,Ruth,ruthba01,1936
2,Honus,Wagner,wagneho01,1936


In [17]:
grader.check("q1a")

q1a results: All test cases passed!

<br><br>

---

## Question 1b
In this question, we will compare the query you wrote in `Question 1a` against the provided query below in `Question 1bi` by inspecting both query plans.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a` by running the cells below.

In [18]:
%%sql --save provided_query result_provided <<
-- just run this cell
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

323 rows affected.

In [19]:
# just run this cell 
provided_query = %sqlcmd snippets provided_query
%sql EXPLAIN ANALYZE {{provided_query.strip(';')}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

8 rows affected.

QUERY PLAN
Nested Loop (cost=0.29..262.79 rows=21 width=278) (actual time=0.033..1.826 rows=323 loops=1)
-> Seq Scan on halloffame (cost=0.00..96.39 rows=21 width=42) (actual time=0.013..0.500 rows=323 loops=1)
Filter: ((inducted)::text = 'Y'::text)
Rows Removed by Filter: 3868
-> Index Scan using master_pkey on people p (cost=0.29..7.92 rows=1 width=274) (actual time=0.004..0.004 rows=1 loops=323)
Index Cond: ((playerid)::text = (halloffame.playerid)::text)
Planning Time: 0.134 ms
Execution Time: 1.858 ms


In [20]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_1a}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

8 rows affected.

QUERY PLAN
Nested Loop (cost=0.29..262.79 rows=21 width=278) (actual time=0.031..1.822 rows=323 loops=1)
-> Seq Scan on halloffame h (cost=0.00..96.39 rows=21 width=42) (actual time=0.012..0.515 rows=323 loops=1)
Filter: ((inducted)::text = 'Y'::text)
Rows Removed by Filter: 3868
-> Index Scan using master_pkey on people p (cost=0.29..7.92 rows=1 width=274) (actual time=0.004..0.004 rows=1 loops=323)
Index Cond: ((playerid)::text = (h.playerid)::text)
Planning Time: 0.117 ms
Execution Time: 1.853 ms


Record the **execution time** and **cost** for each query.

In [21]:
provided_query_cost = 959.06
provided_query_timing = 51.473
your_query_cost = 959.06
your_query_timing = 51.414

In [22]:
grader.check("q1bi")

q1bi results: All test cases passed!

# #### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of all of the below statements that are true.


Consider the following statements:
<br>
A. Both the queries have the same cost
<br>
B. The provided query has a faster execution time because it makes use of a subquery.
<br>
C. The query you wrote has a faster execution time because it does not make use a subquery.
<br>
D. The provided query has less cost because it makes use of a subquery.
<br>
E. The query you wrote has less cost because it does not make use a subquery.
<br>
F. The queries have the same output.
<br>
G. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

In [23]:
q1b_part2 = ['A', 'C', 'F']

In [24]:
grader.check("q1bii")

q1bii results: All test cases passed!

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


# Question 2: Queries and Views, Part 2

In this question, you will continue analyzing queries with/without views and materialized views.
* Question 2a: Write a query that computes people in a Hall of Fame.
* Question 2 Tutorial: Use this query to create a view called `inducted_hof_ca` and a materialized view, `inducted_hof_ca_mat`.
* Question 2b: Write three queries that achieve the same result:
  * Question 2bi: One that uses no views.
  * Question 2bii: One that uses the `inducted_hof_ca` view.
  * Question 2ciii: One that uses the `inducted_hof_ca_mat` materialized view.
* Question 2c: Record the performance of these three queries.
* Question 2d: Analyze and discuss using queries with different types of views.

<br/><br/>

---

## Question 2a

Write a query that returns the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [25]:
%%sql --save query_2a result_2a <<
SELECT p.namefirst, p.namelast, p.playerid, s.schoolid, h.yearid
FROM people AS p
INNER JOIN halloffame AS h ON p.playerid = h.playerid
INNER JOIN collegeplaying AS c ON p.playerid = c.playerid
INNER JOIN schools AS s ON c.schoolid = s.schoolid
WHERE h.inducted  = 'Y' AND s.schoolstate = 'CA'
ORDER BY h.yearid, p.playerid
;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

In [26]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2a = %sqlcmd snippets query_2a
grading_util.save_results("result_2a", query_2a, result_2a)
result_2a

namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [27]:
grader.check("q2a")

q2a results: All test cases passed!

<br/><br/>

---

## Question 2 Tutorial

We are now going to use the query you wrote in the previous part to generate a view, called `inducted_hof_ca`, and a materialized view, `inducted_hof_ca_mat`.

Run the below cells. You do not need to do anything more for this part. 

(Note: the semicolon strip is to avoid executing an empty query with double-semicolons, which causes an error.)

In [28]:
%%sql
/* just run this cell */
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS {{query_2a.strip(';')}};
SELECT * FROM inducted_hof_ca;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


In [29]:
%%sql
/* just run this cell */
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS {{query_2a.strip(';')}};
SELECT * FROM inducted_hof_ca_mat;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

23 rows affected.

namefirst,namelast,playerid,schoolid,yearid
Jackie,Robinson,robinja02,ucla,1962
Harry,Hooper,hoopeha01,stmarysca,1971
Joe,Morgan,morgajo02,camerri,1990
Tom,Seaver,seaveto01,cafrecc,1992
Tom,Seaver,seaveto01,usc,1992
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Ozzie,Smith,smithoz01,calpoly,2002
Tony,Gwynn,gwynnto01,sandiegost,2007


<br/><br/>

---
### Question 2b

For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

You should write three queries that accomplish this task, but with different strategies:
* Question 2bi: Use the `inducted_hof_ca` view;
* Question 2bii Use the `inducted_hof_ca_mat` view; and
* Question 2biii: Do not use `inducted_hof_ca` view, `inducted_hof_ca_mat` materialized view, any common table expressions (CTEs), nor any subqueries.

### Question 2bi

Write a query to accomplish the task above using the `inducted_hof_ca` view. Assign your result to `result_2b_view`.

In [30]:
%%sql --save query_2b_view result_2b_view <<
SELECT schoolid, yearid, count(*)
FROM inducted_hof_ca
GROUP BY schoolid, yearid
ORDER BY yearid ASC

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [31]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_view = %sqlcmd snippets query_2b_view
grading_util.save_results("result_2b_view", query_2b_view, result_2b_view)
result_2b_view

schoolid,yearid,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [32]:
grader.check("q2bi")

q2bi results: All test cases passed!

<br/><br/>

#### Question 2bii:

Now, write the query a second time to use the materialized view `inducted_hof_ca_mat`. Assign your result to `result_2b_mat`.

In [33]:
%%sql --save query_2b_mat result_2b_mat <<
SELECT schoolid, yearid, COUNT(playerid) AS player_count
FROM inducted_hof_ca_mat
GROUP BY schoolid, yearid
ORDER BY yearid ASC;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [34]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_mat = %sqlcmd snippets query_2b_mat
grading_util.save_results("result_2b_mat", query_2b_mat, result_2b_mat)
result_2b_mat

schoolid,yearid,player_count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [35]:
grader.check("q2bii")

q2bii results: All test cases passed!

<br/><br/>

#### Question 2biii:

Finally, write the query a third time to **not** use the `inducted_hof_ca` view, nor the `inducted_hof_ca_mat` materialized view, nor any common table expressions (CTEs), nor any subqueries. Save your result in `result_2b_no_view`.

In [36]:
%%sql --save query_2b_no_view result_2b_no_view <<
SELECT s.schoolid, h.yearid, count(*)
FROM people AS p
INNER JOIN halloffame AS h ON p.playerid = h.playerid
INNER JOIN collegeplaying AS c ON p.playerid = c.playerid
INNER JOIN schools AS s ON c.schoolid = s.schoolid
WHERE h.inducted = 'Y' AND s.schoolstate = 'CA'
GROUP BY s.schoolid, h.yearid
ORDER BY h.yearid ASC

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

13 rows affected.

In [37]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_2b_no_view = %sqlcmd snippets query_2b_no_view
grading_util.save_results("result_2b_no_view", query_2b_no_view, result_2b_no_view)
result_2b_no_view

schoolid,yearid,count
ucla,1962,1
stmarysca,1971,1
camerri,1990,1
cafrecc,1992,1
usc,1992,1
calpoly,2002,4
sandiegost,2007,3
capasad,2008,1
sandiegost,2010,2
calavco,2011,1


In [38]:
grader.check("q2biii")

q2biii results: All test cases passed!

<br/><br/>

---

### Question 2c
Inspect the query plans for the three queries you wrote above by running the following cells.

In [39]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_view}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

27 rows affected.

QUERY PLAN
GroupAggregate (cost=325.73..325.75 rows=1 width=60) (actual time=3.398..3.407 rows=13 loops=1)
"Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid"
-> Sort (cost=325.73..325.74 rows=1 width=52) (actual time=3.392..3.394 rows=23 loops=1)
"Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid"
Sort Method: quicksort Memory: 26kB
-> Subquery Scan on inducted_hof_ca (cost=325.71..325.72 rows=1 width=52) (actual time=3.378..3.384 rows=23 loops=1)
-> Sort (cost=325.71..325.71 rows=1 width=326) (actual time=3.376..3.378 rows=23 loops=1)
"Sort Key: h.yearid, p.playerid"
Sort Method: quicksort Memory: 26kB
-> Nested Loop (cost=0.85..325.70 rows=1 width=326) (actual time=1.152..3.362 rows=23 loops=1)


In [40]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_mat}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

9 rows affected.

QUERY PLAN
Sort (cost=23.67..24.17 rows=200 width=60) (actual time=0.033..0.034 rows=13 loops=1)
Sort Key: yearid
Sort Method: quicksort Memory: 25kB
-> HashAggregate (cost=14.03..16.02 rows=200 width=60) (actual time=0.023..0.026 rows=13 loops=1)
"Group Key: yearid, schoolid"
Batches: 1 Memory Usage: 40kB
-> Seq Scan on inducted_hof_ca_mat (cost=0.00..12.30 rows=230 width=90) (actual time=0.006..0.007 rows=23 loops=1)
Planning Time: 0.075 ms
Execution Time: 0.062 ms


In [41]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_2b_no_view}}

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

23 rows affected.

QUERY PLAN
GroupAggregate (cost=325.71..325.73 rows=1 width=60) (actual time=3.318..3.339 rows=13 loops=1)
"Group Key: h.yearid, s.schoolid"
-> Sort (cost=325.71..325.71 rows=1 width=52) (actual time=3.312..3.315 rows=23 loops=1)
"Sort Key: h.yearid, s.schoolid"
Sort Method: quicksort Memory: 26kB
-> Nested Loop (cost=0.85..325.70 rows=1 width=52) (actual time=1.134..3.298 rows=23 loops=1)
-> Nested Loop (cost=0.57..319.95 rows=19 width=52) (actual time=0.090..2.936 rows=137 loops=1)
-> Nested Loop (cost=0.29..262.79 rows=21 width=80) (actual time=0.034..1.794 rows=323 loops=1)
-> Seq Scan on halloffame h (cost=0.00..96.39 rows=21 width=42) (actual time=0.014..0.508 rows=323 loops=1)
Filter: ((inducted)::text = 'Y'::text)


Then, record the execution time and cost for each query.

In [42]:
with_view_cost = 520.69
with_view_timing = 4.361
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.064
without_view_cost = 519.45
without_view_timing = 4.393

In [43]:
grader.check("q2c")

q2c results: All test cases passed!

<br/><br/>

---

## Question 2d

Given your findings from inspecting the query plans in this Question, as well as your understanding of views and materialized views from lectures, discuss the takeaways of using views and materialized views.

### Question 2di

Assign the variable `q2di` to a list of all of the below statements that are true.

A. Views will reduce the execution time and the cost of a query.<br/>
B. Views will reduce the execution time of a query, but not the cost.<br/>
C. Views will reduce the cost of a query, but not the execution time.<br/>
D. Materialized views reduce the execution time and the cost of a query.<br/>
E. Materialized views reduce the execution time, but not cost of a query<br/>
F. Materialized views reduce the cost of a query, but not the execution time.<br/>
G. Materialized views will result in the same query plan as a query using views.<br/>
H. Materialized views and views take the same time to create.<br/>
I. Materialized views take less time to create than a view.<br/>
J. Materialized views take more time to create than a view.<br/>
    
*Note:* Your answer should look like `q2di = ['A', 'B']`

In [44]:
q2di = ['D', 'J']

In [45]:
grader.check("q2di")

q2di results: All test cases passed!

<!-- BEGIN QUESTION -->

#### Question 2dii:

Explain your answer to the previous part (Question 2di) based on your knowledge from lectures and details from the query plans. Your explanation should also include why you didn't choose certain options. Please answer in maximum 5 sentences.

The primary distinction between a View and a Materialized View is that the latter stores its output on disk, while the former does not. As a result, options 'A', 'B', and 'C' can be dismissed since a view, not being stored on disk, doesn't alter the cost or runtime of a query. In contrast, a materialized view enhances performance by reducing both cost and runtime, as it provides pre-computed data from the disk, eliminating the need to fetch data from scratch. This understanding negates options 'E' and 'F'. Furthermore, since creating a materialized view involves disk I/O operations, it inherently takes longer than creating a regular view, validating option 'J'.

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 3: Predicate Pushdown
In this question, we will explore the impact of predicates (i.e., filters) on a query's execution, particularly inspecting when the optimizer applies predicates.

* Question 3a: Compute a query with all rows.
* Question 3b: Add a simple filter.
* Question 3c: Analyze the tradeoffs to cost.
* Question 3d: Analyze the tradeoffs to execution time.


## Question 3a:
Recall the `inducted_hof_ca` view created in `Question 2`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost using an `EXPLAIN ANALYZE` command.

In [46]:
%%sql
EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

20 rows affected.

QUERY PLAN
Sort (cost=325.71..325.71 rows=1 width=326) (actual time=3.390..3.393 rows=23 loops=1)
"Sort Key: h.yearid, p.playerid"
Sort Method: quicksort Memory: 26kB
-> Nested Loop (cost=0.85..325.70 rows=1 width=326) (actual time=1.157..3.374 rows=23 loops=1)
-> Nested Loop (cost=0.57..319.95 rows=19 width=326) (actual time=0.094..3.032 rows=137 loops=1)
-> Nested Loop (cost=0.29..262.79 rows=21 width=316) (actual time=0.047..1.901 rows=323 loops=1)
-> Seq Scan on halloffame h (cost=0.00..96.39 rows=21 width=42) (actual time=0.013..0.545 rows=323 loops=1)
Filter: ((inducted)::text = 'Y'::text)
Rows Removed by Filter: 3868
-> Index Scan using master_pkey on people p (cost=0.29..7.92 rows=1 width=274) (actual time=0.004..0.004 rows=1 loops=323)


In [47]:
query_view_cost = 528.30
query_view_timing = 4.451

In [48]:
grader.check("q3a")

q3a results: All test cases passed!

<br><br>

---

## Question 3b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [49]:
%%sql
EXPLAIN ANALYZE SELECT * FROM inducted_hof_ca WHERE yearid > 2010;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

20 rows affected.

QUERY PLAN
Sort (cost=185.88..185.89 rows=1 width=326) (actual time=0.839..0.841 rows=8 loops=1)
"Sort Key: h.yearid, p.playerid"
Sort Method: quicksort Memory: 25kB
-> Nested Loop (cost=0.85..185.87 rows=1 width=326) (actual time=0.438..0.828 rows=8 loops=1)
-> Nested Loop (cost=0.57..184.05 rows=6 width=326) (actual time=0.429..0.744 rows=27 loops=1)
-> Nested Loop (cost=0.29..165.00 rows=7 width=316) (actual time=0.385..0.579 rows=31 loops=1)
-> Seq Scan on halloffame h (cost=0.00..106.86 rows=7 width=42) (actual time=0.370..0.408 rows=31 loops=1)
Filter: ((yearid > 2010) AND ((inducted)::text = 'Y'::text))
Rows Removed by Filter: 4160
-> Index Scan using master_pkey on people p (cost=0.29..8.30 rows=1 width=274) (actual time=0.005..0.005 rows=1 loops=31)


In [50]:
query_view_with_filter_cost = 209.85
query_view_with_filter_timing = 0.735

In [51]:
grader.check("q3b")

q3b results: All test cases passed!

<!-- BEGIN QUESTION -->

## Question 3c:
Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose other options). Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows: `A because ...`

**Adding a filter ___ the cost.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

B. decreased because the cost of executing the query with the filter (209.85) is significantly lower than the cost without the filter (528.30). The filter effectively narrows down the number of rows to be processed, leading to a more efficient query execution.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 3d:
Given your findings from inspecting the query plans of queries from Questions 3a and 3b, fill in the blank and **justify your answer**. Explain your answer based on your knowledge from lectures, and details from the query plans (your explanation should include why you didn't choose other options). Your response should be no longer than 3 sentences.

**Note:** Your answer should be formatted as follows: `A because ...`

**Adding a filter ___ the execution time.**
<br>
A. increased
<br>
B. decreased
<br>
C. did not change

B. decreased because the execution time with the filter (0.735 ms) is notably shorter than the execution time without the filter (4.451 ms). By adding the filter, the database processes fewer rows, leading to a faster query execution.

<!-- END QUESTION -->

<br><br>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 4: Join Approaches

In this question, we'll explore different join approaches (Nested Loop Join, Merge Join, Hash Join) and discuss how the query optimizer picks the best approach.

<br/><br/>

---

## Question 4a
Perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In [52]:
%%sql --save query_4a result_4a <<
SELECT * 
FROM people p
INNER JOIN collegeplaying cp ON p.playerid = cp.playerid;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

In [53]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4a = %sqlcmd snippets query_4a
grading_util.save_results("result_4a", query_4a, result_4a);

display(result_4a.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4a}} 

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,height,bats,throws,debut,finalgame,retroid,bbrefid,playerid,schoolid,yearid
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

7 rows affected.

QUERY PLAN
Merge Join (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.035..16.147 rows=17350 loops=1)
Merge Cond: ((p.playerid)::text = (cp.playerid)::text)
-> Index Scan using master_pkey on people p (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.016..3.022 rows=19368 loops=1)
-> Index Only Scan using collegeplaying_pkey on collegeplaying cp (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.013..3.852 rows=17350 loops=1)
Heap Fetches: 17350
Planning Time: 0.125 ms
Execution Time: 16.665 ms


Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4a` to the correct letter choice above, e.g., `q4a = 'A'`.

In [54]:
q4a = 'A'

In [55]:
grader.check("q4a")

q4a results: All test cases passed!

<br><br>

---

## Question 4b

Similar to Question 4a, perform an inner join on the `people` and `collegeplaying` tables on the `playerid` column. Project all columns.

In addition, **sort your output by `playerid`.**

In [56]:
%%sql --save query_4b result_4b <<
SELECT * 
FROM people p
INNER JOIN collegeplaying cp ON p.playerid = cp.playerid
ORDER BY p.playerid;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17350 rows affected.

In [57]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4b = %sqlcmd snippets query_4b
grading_util.save_results("result_4b", query_4b, result_4b);

display(result_4b.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4b}} 

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,height,bats,throws,debut,finalgame,retroid,bbrefid,playerid,schoolid,yearid
0,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,pennst,2001
1,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2002
2,aardsda01,1981,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01,aardsda01,rice,2003


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

7 rows affected.

QUERY PLAN
Merge Join (cost=0.57..3636.67 rows=17350 width=1788) (actual time=0.042..16.638 rows=17350 loops=1)
Merge Cond: ((p.playerid)::text = (cp.playerid)::text)
-> Index Scan using master_pkey on people p (cost=0.29..2298.84 rows=19370 width=1698) (actual time=0.021..3.157 rows=19368 loops=1)
-> Index Only Scan using collegeplaying_pkey on collegeplaying cp (cost=0.29..1072.54 rows=17350 width=90) (actual time=0.016..3.940 rows=17350 loops=1)
Heap Fetches: 17350
Planning Time: 0.137 ms
Execution Time: 17.215 ms


Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4b` to the correct letter choice above, e.g., `q4b = 'A'`.

In [58]:
q4b = 'C'

In [59]:
grader.check("q4b")

q4b results: All test cases passed!

<br><br>

---
## Question 4c
Write a query to retrieve all possible player pair combinations. Project all columns, but **limit to 1000 rows** to ensure your query doesn't take an exorbitant amount of time to run.

**Hint:** You can do this by performing an inner join of the `people` table on itself with an inequality condition.

In [60]:
%%sql --save query_4c result_4c <<
SELECT p1.*, p2.*
FROM people p1
INNER JOIN people p2 ON p1.playerid <> p2.playerid
LIMIT 1000;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

1000 rows affected.

In [61]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_4c = %sqlcmd snippets query_4c
grading_util.save_results("result_4c", query_4c, result_4c);

display(result_4c.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_4c}} 

,playerid,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,deathmonth,deathday,...,namelast,namegiven,weight,height,bats,throws,debut,finalgame,retroid,bbrefid
0,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
1,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
2,aardsda01,1981,12,27,USA,CO,Denver,None,None,None,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

8 rows affected.

QUERY PLAN
Limit (cost=0.00..44.50 rows=1000 width=3396) (actual time=0.015..0.524 rows=1000 loops=1)
-> Nested Loop (cost=0.00..16694169.95 rows=375177530 width=3396) (actual time=0.014..0.469 rows=1000 loops=1)
Join Filter: ((p1.playerid)::text <> (p2.playerid)::text)
Rows Removed by Join Filter: 1
-> Seq Scan on people p1 (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.008..0.008 rows=1 loops=1)
-> Seq Scan on people p2 (cost=0.00..619.70 rows=19370 width=1698) (actual time=0.001..0.103 rows=1001 loops=1)
Planning Time: 0.102 ms
Execution Time: 0.579 ms


Run the cell above to inspect the query plan for your command.

**Which join approach did the query optimizer choose?** 

A. Nested Loop Join<br/>
B. Merge Join<br/>
C. Hash Join<br/>
D. None of the Above

Assign the variable `q4c` to the correct letter choice above, e.g., `q4c = 'A'`.

In [62]:
q4c = 'A'

In [63]:
grader.check("q4c")

q4c results: All test cases passed!

<!-- BEGIN QUESTION -->

<br><br>

---
## Question 4d

Given your findings above, why did the query optimizer ultimately choose the specific join approach you found in each of the above three scenarios in Questions 4a, 4b, and 4c? Feel free to discuss the pros and cons of each join approach as well.

If you feel stuck, here are some things to consider: Does a non-equijoin constrain us to certain join approaches? What's an added benefit in regards to the output of merge join?

**Note:** Your answer should be formatted as follows: `Q4a: A because ... Q4b: A because ...` You should write no more than 5 sentences.

Q4a: The optimizer selected a Hash Join due to the large number of rows and equality-based join condition, leveraging the efficiency of hash tables for such scenarios.

Q4b: A Merge Join was chosen because of the sort requirement on playerid and the pre-sorted output from an index scan.

Q4c: The Nested Loop Join was favored due to the inequality join condition (p1.playerid <> p2.playerid), which is typically handled best by nested loops

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 5: Indexes, Part 1

In Questions 5, 6, and 7, you will analyze how indexes impact query performance.

Question 5:
* Question 5a: Write a query.
* Question 5b: Add an index with a particular index key and reanalyze the previous query's performance.
* Question 5c: Add an index with a different key and reanalyze the previous query's performance.

<br/>

---

## Question 5a
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [64]:
%%sql --save query_5a result_5a <<
SELECT s.playerid, AVG(s.salary) as avg_salary
FROM salaries s
NATURAL JOIN appearances a
WHERE a.g_batting = 10
GROUP BY s.playerid;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

134 rows affected.

In [65]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_5a = %sqlcmd snippets query_5a
grading_util.save_results("result_5a", query_5a, result_5a);

display(result_5a.DataFrame().head(3))
%sql EXPLAIN ANALYZE {{query_5a}} 

,playerid,avg_salary
0,anderla02,240000.0
1,ashbyan01,109000.0
2,ayraubo01,125000.0


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

15 rows affected.

QUERY PLAN
GroupAggregate (cost=3634.44..3634.47 rows=1 width=46) (actual time=17.535..17.590 rows=134 loops=1)
Group Key: s.playerid
-> Sort (cost=3634.44..3634.45 rows=1 width=46) (actual time=17.526..17.535 rows=138 loops=1)
Sort Key: s.playerid
Sort Method: quicksort Memory: 35kB
-> Hash Join (cost=2897.66..3634.43 rows=1 width=46) (actual time=10.962..17.418 rows=138 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=78) (actual time=0.021..1.883 rows=26428 loops=1)
-> Hash (cost=2873.20..2873.20 rows=1223 width=20) (actual time=10.789..10.790 rows=1347 loops=1)
Buckets: 2048 Batches: 1 Memory Usage: 86kB


Inspect the query plan above and record the execution time and cost.

In [66]:
result_5a_cost = 3636.29
result_5a_timing = 24.188

In [67]:
grader.check("q5a")

q5a results: All test cases passed!

<br><br>

---
## Question 5b

Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [68]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
CREATE INDEX g_batting_idx ON appearances(g_batting);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

Now, re-inspect the query plan of the query from `Question 5a` and record its execution time and cost.

In [69]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_5a}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

17 rows affected.

QUERY PLAN
GroupAggregate (cost=2338.44..2338.46 rows=1 width=46) (actual time=8.515..8.582 rows=134 loops=1)
Group Key: s.playerid
-> Sort (cost=2338.44..2338.44 rows=1 width=46) (actual time=8.505..8.513 rows=138 loops=1)
Sort Key: s.playerid
Sort Method: quicksort Memory: 35kB
-> Hash Join (cost=1601.65..2338.43 rows=1 width=46) (actual time=1.921..8.381 rows=138 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=78) (actual time=0.009..1.883 rows=26428 loops=1)
-> Hash (cost=1577.19..1577.19 rows=1223 width=20) (actual time=1.783..1.785 rows=1347 loops=1)
Buckets: 2048 Batches: 1 Memory Usage: 86kB


In [70]:
result_5b_cost =  2369.05
result_5b_timing = 15.711

In [71]:
grader.check("q5b")

q5b results: All test cases passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, **drop the index before moving onto the next question.**

In [72]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

<br><br>

---
## Question 5c

Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table. Make sure to drop the previous index in `Question 5b` first!

In [73]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
DROP INDEX IF EXISTS salary_idx;
CREATE INDEX salary_idx ON salaries(salary);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

Now, re-inspect the query plan of the query from `Question 5a` and record its execution time and cost.

In [74]:
# just run this cell
%sql EXPLAIN ANALYZE {{query_5a}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

15 rows affected.

QUERY PLAN
GroupAggregate (cost=3634.44..3634.47 rows=1 width=46) (actual time=17.639..17.694 rows=134 loops=1)
Group Key: s.playerid
-> Sort (cost=3634.44..3634.45 rows=1 width=46) (actual time=17.630..17.637 rows=138 loops=1)
Sort Key: s.playerid
Sort Method: quicksort Memory: 35kB
-> Hash Join (cost=2897.66..3634.43 rows=1 width=46) (actual time=10.781..17.519 rows=138 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=78) (actual time=0.007..1.959 rows=26428 loops=1)
-> Hash (cost=2873.20..2873.20 rows=1223 width=20) (actual time=10.619..10.620 rows=1347 loops=1)
Buckets: 2048 Batches: 1 Memory Usage: 86kB


In [75]:
result_5c_cost = 3636.29
result_5c_timing = 28.652

In [76]:
grader.check("q5c")

q5c results: All test cases passed!

<br><br>

---

## Question 5d

Given your findings from inspecting the query plans with no indexes (Question 5a), an index on `g_batting` (Question 5b), and an index on `salary` (Question 5c), assign the variable `q5d` to a list of all of the below statements that are true.

A. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.<br/>
B. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.<br/>
C. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.<br/>
D. Adding the `g_batting` index did have a significant impact on the query cost and execution time.<br/>
E. Adding the `salary` index did not have a significant impact on the query execution time and cost.<br/>
F. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.<br/>
G. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.<br/>
H. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q5d = ['A', 'B']`

In [77]:
q5d = ['D', 'E']

In [78]:
grader.check("q5d")

q5d results: All test cases passed!

<!-- BEGIN QUESTION -->

### Question 5di Justification

Explain your answer to `Question 5d` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

The g_batting index notably improved the query's efficiency by enabling a Bitmap Index Scan, eliminating the need for a full scan on the appearances table. In contrast, the salary index didn't enhance the query's performance, as the plan still used a sequential scan on the salaries table, with metrics similar to those without the index.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 6: Indexes, Part 2

Continue the analysis on how indexes impact query performance.

Question 6:
* Question 6a: Write a query that uses an **and** boolean operator. Record query performance.
* Question 6b: Write a query that uses an **or** boolean operator. Record query performances.
* Question 6c: Add an index and rerun queries in Questions 6a, 6b. Record query performance.
* Question 6d: Add a multi-attribute index and rerun queries 6a, 6b. Record query performance.
* Question 6e: Analyze query performance; compare and contrast.

Before continuing, make sure to drop all existing indexes from previous questions.

In [79]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

<br><br>

---

## Question 6a

Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games **and** batted in 10 games (the number of games in which a player played can be found in the `g_all` column of the `appearances` table). Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [80]:
%%sql --save query_6a result_6a <<
SELECT s.playerid, s.yearid, s.salary
FROM salaries s
NATURAL JOIN appearances a
WHERE a.g_all = 10 AND a.g_batting = 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

120 rows affected.

In [81]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6a = %sqlcmd snippets query_6a
grading_util.save_results("result_6a", query_6a, result_6a);

result_6a.DataFrame().head(3)

,playerid,yearid,salary
0,wiggial01,1985,512500.0
1,anderla02,1986,240000.0
2,lakest01,1986,60000.0


In [82]:
grader.check("q6a")

q6a results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [83]:
%sql EXPLAIN ANALYZE {{query_6a}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

8 rows affected.

QUERY PLAN
Nested Loop (cost=0.29..3271.15 rows=1 width=21) (actual time=6.397..11.531 rows=120 loops=1)
-> Seq Scan on appearances a (cost=0.00..3133.84 rows=17 width=20) (actual time=0.012..9.115 rows=1289 loops=1)
Filter: ((g_all = 10) AND (g_batting = 10))
Rows Removed by Filter: 102967
-> Index Scan using salaries_pkey on salaries s (cost=0.29..8.08 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.teamid)::text) AND ((lgid)::text = (a.lgid)::text) AND ((playerid)::text = (a.playerid)::text))
Planning Time: 0.980 ms
Execution Time: 11.564 ms


In [84]:
result_6a_cost = 3287.78
result_6a_timing = 22.796

In [85]:
grader.check("6a_cost")

6a_cost results: All test cases passed!

## Question 6b
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on all the common columns `yearid`, `teamid`, and `playerid`, so feel free to use a natural join.

In [86]:
%%sql --save query_6b result_6b <<
SELECT s.playerid, s.yearid, s.salary
FROM salaries s
NATURAL JOIN appearances a
WHERE a.g_all = 10 OR a.g_batting = 10;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

216 rows affected.

In [87]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_6b = %sqlcmd snippets query_6b
grading_util.save_results("result_6b", query_6b, result_6b);
result_6b.DataFrame().head(3)

,playerid,yearid,salary
0,wiggial01,1985,512500.0
1,forscke01,1986,100000.0
2,carltst01,1986,60000.0


In [88]:
grader.check("q6b")

q6b results: All test cases passed!

Inspect the query plan and record the execution time and cost.

In [89]:
%sql EXPLAIN ANALYZE {{query_6b}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

10 rows affected.

QUERY PLAN
Hash Join (cost=3186.82..3923.59 rows=1 width=21) (actual time=10.298..16.666 rows=216 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=28) (actual time=0.008..1.764 rows=26428 loops=1)
-> Hash (cost=3133.84..3133.84 rows=2649 width=20) (actual time=10.139..10.141 rows=1655 loops=1)
Buckets: 4096 Batches: 1 Memory Usage: 118kB
-> Seq Scan on appearances a (cost=0.00..3133.84 rows=2649 width=20) (actual time=0.007..9.715 rows=1655 loops=1)
Filter: ((g_all = 10) OR (g_batting = 10))
Rows Removed by Filter: 102601
Planning Time: 0.715 ms
Execution Time: 16.704 ms


In [90]:
result_6b_cost = 3927.21
result_6b_timing = 33.210

In [91]:
grader.check("6b_cost")

6b_cost results: All test cases passed!

## Question 6c
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the queries from `Question 6a` and `Question 6b` and record the respective execution costs and times.

In [92]:
%%sql
DROP INDEX IF EXISTS g_batting_idx;
CREATE INDEX g_batting_idx ON appearances(g_batting);


Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [93]:
# record the updated costs for Question 6a ("and" query)
%sql EXPLAIN ANALYZE {{query_6a}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

12 rows affected.

QUERY PLAN
Nested Loop (cost=17.76..1717.26 rows=1 width=21) (actual time=1.891..3.748 rows=120 loops=1)
-> Bitmap Heap Scan on appearances a (cost=17.47..1579.95 rows=17 width=20) (actual time=0.204..1.301 rows=1289 loops=1)
Recheck Cond: (g_batting = 10)
Filter: (g_all = 10)
Rows Removed by Filter: 58
Heap Blocks: exact=899
-> Bitmap Index Scan on g_batting_idx (cost=0.00..17.46 rows=1223 width=0) (actual time=0.112..0.112 rows=1347 loops=1)
Index Cond: (g_batting = 10)
-> Index Scan using salaries_pkey on salaries s (cost=0.29..8.08 rows=1 width=28) (actual time=0.002..0.002 rows=0 loops=1289)
Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.teamid)::text) AND ((lgid)::text = (a.lgid)::text) AND ((playerid)::text = (a.playerid)::text))


In [94]:
result_6cand_index_cost = 1762.87
result_6cand_index_timing = 6.437

In [95]:
# record the updated costs for Question 6b ("or" query)
%sql EXPLAIN ANALYZE {{query_6b}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

10 rows affected.

QUERY PLAN
Hash Join (cost=3186.82..3923.59 rows=1 width=21) (actual time=10.853..17.218 rows=216 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=28) (actual time=0.005..1.756 rows=26428 loops=1)
-> Hash (cost=3133.84..3133.84 rows=2649 width=20) (actual time=10.700..10.702 rows=1655 loops=1)
Buckets: 4096 Batches: 1 Memory Usage: 118kB
-> Seq Scan on appearances a (cost=0.00..3133.84 rows=2649 width=20) (actual time=0.023..10.232 rows=1655 loops=1)
Filter: ((g_all = 10) OR (g_batting = 10))
Rows Removed by Filter: 102601
Planning Time: 0.657 ms
Execution Time: 17.254 ms


In [96]:
result_6cor_index_cost = 3927.21
result_6cor_index_timing = 34.547

In [97]:
grader.check("q6c")

q6c results: All test cases passed!

<br/><br/>

---

## Question 6d: Multiple-attribute index

Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost for the "or" command in `Question 6b`.

Before continuing, make sure to drop all existing indexes from previous questions.

In [98]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [99]:
%%sql 
DROP INDEX IF EXISTS g_batting_all_idx;
CREATE INDEX g_batting_g_all_idx ON appearances(g_batting, g_all);

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

++
||
++
++

In [100]:
# record the updated costs for Question 6b ("or" query)
%sql EXPLAIN ANALYZE {{query_6b}} 

Running query in 'postgresql://jovyan@127.0.0.1:5432/baseball'

15 rows affected.

QUERY PLAN
Hash Join (cost=2896.05..3632.83 rows=1 width=21) (actual time=2.806..9.209 rows=216 loops=1)
Hash Cond: ((s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text) AND ((s.lgid)::text = (a.lgid)::text) AND ((s.playerid)::text = (a.playerid)::text))
-> Seq Scan on salaries s (cost=0.00..459.28 rows=26428 width=28) (actual time=0.005..1.574 rows=26428 loops=1)
-> Hash (cost=2843.07..2843.07 rows=2649 width=20) (actual time=2.632..2.635 rows=1655 loops=1)
Buckets: 4096 Batches: 1 Memory Usage: 118kB
-> Bitmap Heap Scan on appearances a (cost=1181.00..2843.07 rows=2649 width=20) (actual time=0.925..2.237 rows=1655 loops=1)
Recheck Cond: ((g_all = 10) OR (g_batting = 10))
Heap Blocks: exact=1027
-> BitmapOr (cost=1181.00..1181.00 rows=2665 width=0) (actual time=0.810..0.811 rows=0 loops=1)
-> Bitmap Index Scan on g_batting_g_all_idx (cost=0.00..1162.21 rows=1442 width=0) (actual time=0.766..0.767 rows=1597 loops=1)


In [101]:
result_6d_multiple_col_index_cost = 3621.73
result_6d_multiple_col_index_timing = 17.287

In [102]:
grader.check("q6d")

q6d results: All test cases passed!

<br/><br/>

---

## Question 6e
Given your findings from inspecting the query plans from all parts of this `Question 6`, assign the variable `q6e` to a list of all below statements that are true.

A. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.<br/>
B. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.<br/>
C. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.<br/>
D. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.<br/>
E. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.<br/>
F. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.<br/>
G. Adding a multicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.<br/>
H. Adding a multicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.<br/>
I. Adding a multicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q6e = ['A', 'B']`


In [103]:
q6e = ['C', 'G']

In [ ]:
grader.check("q6e")

<!-- BEGIN QUESTION -->

### Question 6ei Justification

Explain your answer to `Question 6e` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

Adding an index on g_batting for an AND predicate reduces both execution time and cost due to efficient data retrieval in the B+ tree structure, eliminating options 'A' and 'B'. Indexing a column in an OR predicate doesn't guarantee efficiency since all records must be verified for the second condition, ruling out "D", "E", and "F". A multicolumn index on g_batting and g_all optimizes execution time, especially when the query involves the leftmost column, g_batting, making 'G' the correct choice.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 7: Indexes, Part 3

Continue the analysis on how indexes impact query performance. Now, use aggregators.

Question 7:
* Question 7a: Write two queries that use aggregators. Record query performance.
* Question 7b: Add an index and rerun queries in Questions 7a. Record query performance.
* Question 7c: Analyze query performance; compare and contrast.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;

---

## Question 7a

Write 2 queries: one that finds the minimum salary from the salary table `Salaries` and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

Minimum salary:

In [ ]:
%%sql --save query_7a_min result_7a_min << 
SELECT MIN(salary) AS min_salary
FROM salaries;

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7a_min = %sqlcmd snippets query_7a_min
grading_util.save_results("result_7a_min", query_7a_min, result_7a_min);

display(result_7a_min)
%sql EXPLAIN ANALYZE {{query_7a_min}} 

In [ ]:
result_7a_min_query_cost = 525.36
result_7a_min_query_timing = 3.472

In [ ]:
grader.check("q7a_min")

Average salary:

In [ ]:
%%sql --save query_7a_avg result_7a_avg <<
SELECT AVG(salary) AS avg_salary
FROM salaries;

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_7a_avg = %sqlcmd snippets query_7a_avg
grading_util.save_results("result_7a_avg", query_7a_avg, result_7a_avg);

display(result_7a_avg)
%sql EXPLAIN ANALYZE {{query_7a_avg}} 

In [ ]:
result_7a_avg_query_cost = 525.36
result_7a_avg_query_timing = 7.777

In [ ]:
grader.check("q7a_avg")

<br><br>

---
## Question 7b
Create an index on the `salary` column in the `Salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [ ]:
%%sql 
DROP INDEX IF EXISTS salary_idx;
CREATE INDEX salary_idx ON salaries(salary);


In [ ]:
# record the updated costs for "min" query
%sql EXPLAIN ANALYZE {{query_7a_min}} 

In [ ]:
result_7b_min_query_cost = 0.33
result_7b_min_query_timing = 0.104

In [ ]:
# record the updated costs for "avg" query
%sql EXPLAIN ANALYZE {{query_7a_avg}} 

In [ ]:
result_7b_avg_query_cost = 525.36
result_7b_avg_query_timing = 7.356

In [ ]:
grader.check("q7b")

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 7c
Given your findings from `Question 7`, which of the following statements is true?
<br> A. An index on the column being aggregated in a query will always provide a performance enhancement.
<br> B. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
<br> C. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
<br> D. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

**Justify your answer.** Explain your answer based on your knowledge from lectures, and details of the query plans (your explanation should include why you didn't choose certain options). Your response should be no longer than 3 sentences.
 
*Note:* Your answer should be formatted as follows: "A because ... " 

D because the query finding the MIN(salary) benefited from the index on the salary column, as evidenced by the shift from a sequential scan to an index-only scan, significantly reducing execution time and cost. However, the query finding AVG(salary) still relied on a sequential scan, showing no performance improvement with the index. This suggests that while MIN and MAX operations can leverage indexes for optimization, AVG and COUNT operations might not always benefit in the same way.

<!-- END QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 8: Clustered Indexes
In this question, we will inspect the impact that clustering our data on an index can have on a query's performance.
* Question 8a: query
* Question 8b: cluster index on primary key
* Question 8c: cluster index on different key
* Question 8d: observe and analyze.

Before continuing, make sure to drop all existing indexes from previous questions.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;

---

## Question 8a

Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [ ]:
%%sql --save query_8a result_8a <<
SELECT playerid, yearid, teamid, ab
FROM batting
WHERE ab > 500;

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8a = %sqlcmd snippets query_8a
grading_util.save_results("result_8a", query_8a, result_8a);
result_8a.DataFrame().head(3)

In [ ]:
grader.check("q8a")

Inspect the query plan and record the execution time and cost.

In [ ]:
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8a_cost = 2884.05
result_8a_timing = 9.861

In [ ]:
grader.check("8a_cost")

<br><br>

---

## Question 8b

Cluster the `batting` table on its primary key (hint: use the psql meta-command `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it.

Then, re-inspect the query plan for the query from `Question 8a` and record the execution time and cost.

In [ ]:
%%sql
CLUSTER batting USING batting_pkey;


In [ ]:
# check the updated costs for query in Question 8a
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8b_cost = 2878.05
result_8b_timing = 20.692

In [ ]:
grader.check("q8b")

<br><br>

---

## Question 8c

Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table AND cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [ ]:
%%sql --save query_8c result_8c <<
CREATE INDEX ab_idx ON batting(ab);
CLUSTER batting USING ab_idx;

In [ ]:
# Do not delete/edit this cell!
# You must run this cell before running the autograder.
query_8c = %sqlcmd snippets query_8c
grading_util.save_results("result_8c", query_8c, result_8c);

# check the updated costs for query in Question 8a
%sql EXPLAIN ANALYZE {{query_8a}} 

In [ ]:
result_8c_cost = 1788.39
result_8c_timing = 3.308

In [ ]:
grader.check("q8c")

<br><br>

---

## Question 8d
Given your findings from inspecting the query plans from Questions 8a, 8b, and 8c, assign the variable `q8d` to a list of all statements that are true.

A. Clustering based on the `ab_idx` decreased the cost of the query.<br/>
B. Clustering based on the `ab_idx` increased the cost of the query.<br/>
C. Clustering based on the `ab_idx` increased the execution time of the query.<br/>
D. Clustering based on the `ab_idx` decreased the execution time of the query.<br/>
E. Clustering based on the `batting_pkey` decreased the cost of the query.<br/>
F. Clustering based on the `batting_pkey` increased the cost of the query.<br/>
G. Clustering based on the `batting_pkey` increased the execution time of the query.<br/>
H. Clustering based on the `batting_pkey` decreased the execution time of the query.<br/>
I. None of the above
    
**Note:** Your answer should be formatted as a list of single-character strings, e.g., `q8d = ['A', 'B']`.


In [ ]:
q8d = ['A', 'D']

In [ ]:
grader.check("q8d")

<br><br>

---

### Question 8di Justification

Explain your answer to `Question 8d` above based on your knowledge from lectures, and details from inspecting the query plans (your explanation should include why you didn't choose certain options). Your answer should be no longer than 3 sentences.

Clustering based on the ab_idx led to a notable reduction in both the query cost and execution time, primarily because the query's WHERE predicate is on the 'ab' column, which was indexed and clustered. This optimization made data retrieval more efficient. In contrast, clustering on the batting_pkey did not impact the query's cost or execution time since the WHERE predicate isn't based on the batting column, ruling out options E, F, G, H, and I.

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 9: Cost of Index Management
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

Before starting this question, be sure to delete any indexes by running the below cell.

In [ ]:
# you must run this cell!!!
%sql DROP INDEX IF EXISTS g_batting_idx;
%sql DROP INDEX IF EXISTS salary_idx;
%sql DROP INDEX IF EXISTS g_batting_all_idx;
%sql DROP INDEX IF EXISTS ab_idx;

---

## Question 9a

Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [ ]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

Next, run the provided update script and record the **wall time**.

**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of this subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_9a_timing = 2.85

In [ ]:
grader.check("q9a")

<br/><br/>

**Before moving onto the next question**,  delete all the rows that were added to the table from the update script.

In [ ]:
%%sql
/* just run this cell */
DELETE FROM salaries
WHERE added = 'true';

<br><br>

---

## Question 9b

Now, create an index on the `salary` column and record the **wall time** after executing the update script. Make sure to first run the previous cell to rollback any changes from the previous part!

In [ ]:
%%sql 
CREATE INDEX salary_idx ON salaries(salary);


**NOTE:** Running the below cell multiple times may result in an error, unless you first delete the rows with the cell given at the end of last subpart.

In [ ]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

In [ ]:
result_9b_timing = 4.93

In [ ]:
grader.check("q9b")

<!-- BEGIN QUESTION -->

<br><br>

---

## Question 9c:
What difference did you notice when you added an index into the salaries table and re-timed the update? Why do you think it happened? Your answer should be no longer than 3 sentences.

After adding an index, the insertion time noticeably increased. This is likely because every new data insertion requires the database to maintain the order of the indexed values. For instance, if a new value needs to be placed between existing indexed data, the system must first insert the new data and then adjust subsequent indexed values accordingly, leading to additional time overhead.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<br/><br/><br/>

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

# Question 10: Project Takeaways

In this project, we explored how the database system optimizes query execution and how users can futher tune the performance of their queries.

Familiarizing yourself with these optimization and tuning methods will make you a better data engineer. In this question, we'll ask you to recall and summarize these concepts. Who knows? Maybe one day it will help you during an interview or on a project.

In the following answer cell,
1. Name 3 methods you learned in this project. The method can be either the optimization done by the database system, or the fine tuning done by the user.
2. For each method, summarize how and why it can optimize query performance. Feel free to discuss any drawbacks, if applicable.

Your answer should be no longer than ten sentences. Each method identification/discussion is 2 points.


Index Creation and Utilization: Through this project, I grasped the transformative impact of indexes on query performance. Without an index, the database performs a sequential scan, sifting through each row to find the desired data. However, with an indexed column, the data is organized in a B+tree structure, allowing for direct and efficient access to the required rows. This structured arrangement ensures that rows with similar index values are adjacent, drastically reducing query cost and execution time.

Subqueries and Their Impact: I discovered that while subqueries can be powerful, they often come at a performance cost. Each subquery essentially reads datasets from the beginning, increasing the overall execution time. To optimize, it's beneficial to consider alternatives like using the 'WITH' clause or joining tables on essential columns. The key takeaway is the importance of minimizing the size of the datasets being worked on, as larger datasets inherently demand more resources.

Understanding Predicate Effects on Indexing: The project highlighted the nuanced relationship between query predicates ("OR" vs. "AND") and index effectiveness. It's crucial to recognize that an index might not always optimize performance, especially in queries with "OR" predicates. Being aware of these intricacies ensures that we apply indexing judiciously, maximizing its benefits.

<!-- END QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## Congratulations! You have finished Project 2.

Run the following cell to zip and download the results of your queries. You will also need to run the export cell at the end of the notebook.

**Please save your notebook before exporting (this is a good time to do it!)** Otherwise, we may not be able to register your written responses.

**For submission on Gradescope, you will need to submit BOTH the `proj2.zip` file generated by the export cell and the `results.zip` file generated by the following cell.**

**Common submission issues:** You MUST submit the generated zip files (not folders) to the autograder. However, Safari is known to automatically unzip files upon downloading. You can fix this by going into Safari preferences, and deselect the box with the text "Open safe files after downloading" under the "General" tab. If you experience issues with downloading via clicking on the link, you can also navigate to the project 2 directory within JupyterHub (remove `proj2.ipynb` from the url), and manually download the generated zip files. Please post on Ed if you encounter any other submission issues.

In [ ]:
grading_util.prepare_submission_and_cleanup()  # builds results.zip

In [ ]:
# Close SQL magic connection
# You may disregard "RunTimeError: Could not close connection"
# %sql --close postgresql://127.0.0.1:5432/baseball

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True, files=['results.zip'])